In [1]:
import pandas as pd
from astropy.time import Time

In [2]:
longitude = 107.6
latitude = -6.8
times = '2017-09-12T00:00:00' #midnight UTC, for example input 2017-09-12 to output 2017-09-11 prediction

maxMag = 12
maxDur = 200

In [3]:
t = Time(times, format='isot', scale='utc')
url = "http://var2.astro.cz/ETD/predictions.php?JDmidnight={:f}&delka={:f}&sirka={:f}".format(t.jd, longitude, latitude)
dfs = pd.read_html(url)
a = dfs[2][pd.notnull(dfs[2][0])]
a = a.reset_index(drop=True)
a.columns = a.iloc[0]
a = a[1:]
a = a.assign(beginUT=a['BEGIN(UT/h,A)'][:].str[:5])
a = a.assign(rasi=a['OBJECT'][:].str[-3:])
a['OBJECT'] = a['OBJECT'][:].str[:-3]
a = a.join(a['BEGIN(UT/h,A)'].str.split(',', 1, expand=True).rename(columns={0:'AltBegin', 1:'DirBegin'}))
a['AltBegin'] = a['AltBegin'][:].str[-3:]
a = a.drop('BEGIN(UT/h,A)', 1)
a = a.assign(Center=a['CENTER(DD.MM. UT/h,A)'][:].str[7:12])
a = a.join(a['CENTER(DD.MM. UT/h,A)'].str.split(',', 1, expand=True).rename(columns={0:'AltCen', 1:'DirCen'}))
a['AltCen'] = a['AltCen'][:].str[12:]
a = a.drop('CENTER(DD.MM. UT/h,A)', 1)
a = a.assign(End=a['END(UT/h,A)'][:].str[:5])
a = a.join(a['END(UT/h,A)'].str.split(',', 1, expand=True).rename(columns={0:'AltEnd', 1:'DirEnd'}))
a['AltEnd'] = a['AltEnd'][:].str[5:]
a = a.drop('END(UT/h,A)', 1)
a = a.join(a['ElementsCoords'].str.split(':', 2, expand=True).rename(columns={1:'RA', 2:'DEC'}))
a = a.drop('ElementsCoords', 1)
a = a.drop(0, 1)
a['RA'] = a['RA'][:].str[:13]
a['DEC'] = a['DEC'][:].str[1:]
a[['D(min)', 'V(MAG)', 'DEPTH(MAG)']] = a[['D(min)', 'V(MAG)', 'DEPTH(MAG)']].apply(pd.to_numeric)

In [4]:
a

,OBJECT,D(min),V(MAG),DEPTH(MAG),beginUT,rasi,AltBegin,DirBegin,Center,AltCen,DirCen,End,AltEnd,DirEnd,RA,DEC
1,HD209458 b,184.200,7.650,0.0162,10:40,Peg,15°,E,12:12,36°,NE,13:44,55°,NE,22 03 10DE,+18 53 04
2,WASP-112 b,172.610,13.300,0.0161,10:48,PsA,13°,SE,12:14,32°,SE,13:40,48°,SE,22 37 57.43D,-35 09 13.9
3,HATS-24 b,145.200,12.830,0.0183,11:47,Pav,37°,S,12:59,34°,S,14:12,28°,SW,17 55 33.60D,-61 44 50.3
4,Kepler-6 b,240.000,13.800,0.0113,11:33,Cyg,31°,N,13:33,35°,N,15:33,27°,NW,19 47 20.9DE,+48 14 23.8
5,CoRoT-6 b,245.000,13.900,0.0151,11:32,Oph,74°,NE,13:35,65°,NW,15:37,36°,W,18 44 17.42D,06 39 47.95
6,HATS-34 b,93.300,13.849,0.0242,13:00,Tuc,18°,SE,13:46,23°,SE,14:33,27°,S,00 03 05.88D,-62 28 09.6
7,WASP-8 b,264.400,9.790,0.0151,11:58,Scl,11°,SE,14:10,38°,SE,16:22,59°,SE,23 59 36.07D,-35 01 52.9
8,HAT-P-6 b,202.800,10.500,0.0094,12:47,And,15°,NE,14:28,30°,NE,16:10,40°,N,23 39 05.85D,+42 27 57.5
9,HATS-11 b,261.900,14.018,0.0125,13:30,Sgr,71°,SW,15:41,44°,SW,17:52,14°,W,19 17 36.24D,-22 23 23.7
10,CoRoT-11 b,150.054,12.940,0.0123,14:49,Ser,48°,W,16:04,30°,W,17:19,11°,W,18 42 44.95D,05 56 16.12


In [5]:
a[(a['V(MAG)']<maxMag) & (a['D(min)']<maxDur)]

,OBJECT,D(min),V(MAG),DEPTH(MAG),beginUT,rasi,AltBegin,DirBegin,Center,AltCen,DirCen,End,AltEnd,DirEnd,RA,DEC
1,HD209458 b,184.200,7.65,0.0162,10:40,Peg,15°,E,12:12,36°,NE,13:44,55°,NE,22 03 10DE,+18 53 04
14,WASP-33 b,163.000,8.30,0.0151,16:30,And,26°,NE,17:51,38°,NE,19:13,45°,N,02 26 51.08D,+37 33 02.5
16,KELT-1 b,153.245,10.70,0.0066,17:36,And,44°,N,18:53,40°,N,20:09,30°,NW,00 01 26.92D,39 23 01.7
17,WASP-50 b,108.400,11.55,0.0215,19:25,Eri,78°,E,20:19,88°,S,21:14,76°,W,02 54 45.14D,-10 53 52.93
18,WASP-32 b,145.000,11.30,0.0135,19:33,Psc,61°,W,20:45,43°,W,21:58,25°,W,00 15 50.81D,+01 12 02.0
